In [5]:
# Part of agent.py --> Follow https://google.github.io/adk-docs/get-started/quickstart/ to learn the setup

import asyncio
import os
from google.adk.agents import LoopAgent, LlmAgent, BaseAgent, SequentialAgent
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.agents.invocation_context import InvocationContext
from google.adk.tools.tool_context import ToolContext
from typing import AsyncGenerator, Optional
from google.adk.events import Event, EventActions

# --- Constants ---
APP_NAME = "doc_writing_app_v3"  # New App Name
USER_ID = "dev_user_01"
SESSION_ID_BASE = "loop_exit_tool_session"  # New Base Session ID
GEMINI_MODEL = "gemini-2.0-flash"
STATE_INITIAL_TOPIC = "initial_topic"

# --- State Keys ---
STATE_CURRENT_DOC = "current_document"
STATE_CRITICISM = "criticism"
# Define the exact phrase the Critic should use to signal completion
COMPLETION_PHRASE = "No major issues found."


# --- Tool Definition ---
def exit_loop(tool_context: ToolContext):
    """Call this function ONLY when the critique indicates no further changes are needed, signaling the iterative process should end."""
    print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
    tool_context.actions.escalate = True
    # Return empty dict as tools should typically return JSON-serializable output
    return {}


# --- Agent Definitions ---

# STEP 1: Initial Writer Agent (Runs ONCE at the beginning)
initial_writer_agent = LlmAgent(
    name="InitialWriterAgent",
    model=GEMINI_MODEL,
    include_contents="none",
    # MODIFIED Instruction: Ask for a slightly more developed start
    instruction=f"""あなたは物語を始めるためのクリエイティブライティングアシスタントです。
    短編小説の*最初の草稿*を書いてください（2-4文を目指してください）。
    以下のトピックに基づいて*のみ*内容を作成してください。魅力的にするために、具体的な要素（キャラクター、設定の詳細、開始アクションなど）を導入してみてください。
    トピック: {{initial_topic}}

    物語/ドキュメントのテキスト*のみ*を出力してください。導入や説明は追加しないでください。
""",
    description="トピックに基づいて初期ドキュメントの草稿を作成し、ある程度の初期内容を目指します。",
    output_key=STATE_CURRENT_DOC,
)

# STEP 2a: Critic Agent (Inside the Refinement Loop)
critic_agent_in_loop = LlmAgent(
    name="CriticAgent",
    model=GEMINI_MODEL,
    include_contents="none",
    # MODIFIED Instruction: More nuanced completion criteria, look for clear improvement paths.
    instruction=f"""あなたは短いドキュメントの草稿（通常2-6文）をレビューする建設的な批評AIです。あなたの目標はバランスの取れたフィードバックです。

    **レビューするドキュメント:**
    ```
    {{current_document}}
    ```

    **タスク:**
    初期トピック（既知の場合）に従って、ドキュメントの明確性、魅力度、基本的な一貫性をレビューしてください。

    トピックをより良く捉えたり、読者の関心を高めたりするために、ドキュメントを改善できる1-2つの*明確で実行可能な*方法を特定した場合（例：「より強い冒頭文が必要」、「キャラクターの目標を明確にする」）:
    これらの具体的な提案を簡潔に提供してください。批評テキスト*のみ*を出力してください。

    それ以外の場合、ドキュメントが一貫性があり、その長さに対してトピックに適切に対応し、明らかなエラーや明白な欠落がない場合:
    *正確に*「{COMPLETION_PHRASE}」というフレーズで応答し、それ以外は何も出力しないでください。完璧である必要はありませんが、この段階では機能的に完了しているだけで十分です。核となる部分が健全な場合、純粋に主観的な文体の好みを提案することは避けてください。

    説明は追加しないでください。批評または正確な完了フレーズのみを出力してください。
""",
    description="現在の草稿をレビューし、明確な改善が必要な場合は批評を提供し、そうでない場合は完了を合図します。",
    output_key=STATE_CRITICISM,
)


# STEP 2b: Refiner/Exiter Agent (Inside the Refinement Loop)
refiner_agent_in_loop = LlmAgent(
    name="RefinerAgent",
    model=GEMINI_MODEL,
    # Relies solely on state via placeholders
    include_contents="none",
    instruction=f"""あなたはフィードバックに基づいてドキュメントを改善する、またはプロセスを終了するクリエイティブライティングアシスタントです。
    **現在のドキュメント:**
    ```
    {{current_document}}
    ```
    **批評/提案:**
    {{criticism}}

    **タスク:**
    '批評/提案'を分析してください。
    批評が*正確に*「{COMPLETION_PHRASE}」の場合:
    'exit_loop'関数を呼び出す必要があります。テキストは出力しないでください。
    それ以外の場合（批評に実行可能なフィードバックが含まれている場合）:
    提案を慎重に適用して「現在のドキュメント」を改善してください。改善されたドキュメントのテキスト*のみ*を出力してください。

    説明は追加しないでください。改善されたドキュメントを出力するか、exit_loop関数を呼び出してください。
""",
    description="批評に基づいてドキュメントを改善するか、批評が完了を示している場合はexit_loopを呼び出します。",
    tools=[exit_loop],  # Provide the exit_loop tool
    output_key=STATE_CURRENT_DOC,  # Overwrites state['current_document'] with the refined version
)


# STEP 2: Refinement Loop Agent
refinement_loop = LoopAgent(
    name="RefinementLoop",
    # Agent order is crucial: Critique first, then Refine/Exit
    sub_agents=[
        critic_agent_in_loop,
        refiner_agent_in_loop,
    ],
    max_iterations=5,  # Limit loops
)

# STEP 3: Overall Sequential Pipeline
# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = SequentialAgent(
    name="IterativeWritingPipeline",
    sub_agents=[
        initial_writer_agent,  # Run first to create initial doc
        refinement_loop,  # Then run the critique/refine loop
    ],
    description="初期ドキュメントを作成し、終了ツールを使用して批評で反復的に改善します。",
)